In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

iris_path = "iris.csv"
iris_data = pd.read_csv(iris_path)

iris_features = iris_data.drop(columns=["species"]).values #elimin targetul

n_clusters = 3
n_features = iris_features.shape[1]

with pm.Model() as gmm_model:
    weights = pm.Dirichlet("weights", a=np.ones(n_clusters))
    means = pm.Normal("means", mu=0, sigma=10, shape=(n_clusters, n_features))
    sds = pm.Exponential("sds", lam=1, shape=(n_clusters, n_features))
    cluster_assignments = pm.Categorical(
        "cluster_assignments", p=weights, shape=iris_features.shape[0]
    )
    likelihood = pm.Normal(
        "observations",
        mu=means[cluster_assignments],
        sigma=sds[cluster_assignments],
        observed=iris_features,
    )
    trace = pm.sample(150, tune=500, random_seed=42, return_inferencedata=True)

cluster_assignments = trace.posterior["cluster_assignments"].mean(axis=0).argmax(axis=-1).values

iris_data["inferred_cluster"] = cluster_assignments

print(iris_data.head())

az.plot_trace(trace)
az.summary(trace, var_names=["weights", "means", "sds"])


In [ ]:
import pandas as pd
import pymc as pm
import numpy as np

data = pd.read_csv('date_alegeri_turul2.csv')

X = data[['Varsta', 'Sex', 'Educatie', 'Venit']].values
y = data['Vot'].values

X_normalized = (X - X.mean(axis=0)) / X.std(axis=0)

with pm.Model() as logistic_model:
    beta = pm.Normal("beta", mu=0, sigma=10, shape=X_normalized.shape[1])
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    p = pm.math.sigmoid(intercept + pm.math.dot(X_normalized, beta))
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)
    trace = pm.sample(1000, tune=1000, return_inferencedata=True, random_seed=42)

import arviz as az

summary = az.summary(trace, var_names=["beta", "intercept"])
print(summary)

az.plot_forest(trace, var_names=["beta", "intercept"])
